In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Setup browser
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Load page
driver.get("https://www.digikala.com/")
time.sleep(5)

soup = BeautifulSoup(driver.page_source, 'html.parser')

# Create list to hold [text, href]
linked_data = []

for a in soup.find_all('a', href=True):
    text = a.get_text(strip=True)
    href = a['href']
    if text:
        linked_data.append([text, href])
    if len(linked_data) >= 100:  # Stop after 100 entries
        break

# Print the result
print("\n--- Linked Text and URLs (first 100) ---\n")
for item in linked_data:
    print(item)

driver.quit()



--- Linked Text and URLs (first 100) ---

['سوپرمارکت', '/main/food-beverage/']
['پرفروش\u200cترین\u200cها', '/best-selling/']
['خرید کالای کارکرده', 'https://www.digikala.com/landing/used/?utm_source=DKHEADER&utm_medium=HEADERBTM&utm_campaign=DKHEADER-USED&utm_id=DK-HEADER-USEDLANDING']
['شگفت\u200cانگیزها', '/incredible-offers/']
['سوالی دارید؟', '/faq/']
['در دیجی\u200cکالا بفروشید', '/landings/seller-introduction/']
['همه موارد این دسته', '/search/category-mobile-phone/']
['انتخاب موبایل', '/landing/mobile/']
['همه موارد این دسته', '/search/category-mobile-phone/apple/']
['آیفون ۱۶', '/tags/iphone-16/']
['آیفون ۱۶ ای', '/product/dkp-19656793/%DA%AF%D9%88%D8%B4%DB%8C-%D9%85%D9%88%D8%A8%D8%A7%DB%8C%D9%84-%D8%A7%D9%BE%D9%84-%D9%85%D8%AF%D9%84-iphone-16e-hn-%D8%AA%DA%A9-%D8%B3%DB%8C%D9%85-%DA%A9%D8%A7%D8%B1%D8%AA-%D8%B8%D8%B1%D9%81%DB%8C%D8%AA-128-%DA%AF%DB%8C%DA%AF%D8%A7%D8%A8%D8%A7%DB%8C%D8%AA-%D9%88-%D8%B1%D9%85-8-%DA%AF%DB%8C%DA%AF%D8%A7%D8%A8%D8%A7%DB%8C%D8%AA']
['آیفون ۱۶ پرو', 

In [ ]:
import stanza
# import spacy 
# Download Persian model (run once)
# stanza.download('fa')

# Initialize the pipeline for Persian
nlp = stanza.Pipeline('fa')

# Your Persian text
text = "قیمت تلفن امروز 20 است فردا ارزان میشود "

# Process the text
doc = nlp(text)

# Extract keywords (nouns and proper nouns)
keywords = []
for sentence in doc.sentences:
    for word in sentence.words:
        if word.upos in ['NOUN', 'PROPN']:
            keywords.append(word.text)

# Remove duplicates
unique_keywords = list(set(keywords))

print("Keywords:", unique_keywords)

c:\Users\MM\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-27 17:04:52 INFO: Downloaded file to C:\Users\MM\stanza_resources\resources.json
2025-07-27 17:04:52 INFO: Downloading default packages for language: fa (Persian) ...
2025-07-27 17:04:52 INFO: File exists: C:\Users\MM\stanza_resources\fa\default.zip
2025-07-27 17:04:54 INFO: Finished downloading models and saved to C:\Users\MM\stanza_resources
2025-07-27 17:04:54 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-07-27 17:04:55 INFO: Downloaded file to C:\Users\MM\stanza_resources\resources.json
2025-07-27 17:04:56 INFO: Loading these models for language: fa (Per

Keywords: ['20', 'قیمت', 'تلفن', 'فردا', 'امروز']


In [2]:
from sentence_transformers import SentenceTransformer, util
import stanza

# Load models
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
stanza.download('fa')  # Only needs to be done once
nlp = stanza.Pipeline('fa')

# Assume linked_data is a list of [text, href] pairs
cut = 100
linked_data_cut = unique_keywords[:cut]  # Use top 100 items

# Prepare keyword embedding
words = [item[0] for item in linked_data_cut]
keyword_text = " ".join(words)
keyword_embedding = model.encode(keyword_text, convert_to_tensor=True)

# Preprocess with Stanza
doc = nlp(keyword_text)
sentences = [" ".join([word.text for word in sentence.words]) for sentence in doc.sentences]

# Match sentences with their original links
SIMILARITY_THRESHOLD = 0.6
matched_sentences = []

for text, href in linked_data:
    if not text.strip():
        continue
    sent_embedding = model.encode(linked_data, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(keyword_embedding, sent_embedding).item()
    if similarity >= SIMILARITY_THRESHOLD:
        matched_sentences.append((text, href, similarity))

# Sort and print results
matched_sentences = sorted(matched_sentences, key=lambda x: x[2], reverse=True)

print("\nجملات مرتبط:")
for text, href, score in matched_sentences:
    print(f"{text}\n🔗 لینک: {href}\n")


OSError: The paging file is too small for this operation to complete. (os error 1455)